In [1]:
import gdown
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from osgeo import gdal

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
import torchvision.transforms.functional as vF

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, jaccard_score, hamming_loss, label_ranking_loss, coverage_error, classification_report
import sklearn.metrics as metr

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size=32

In [2]:
data_url = "https://zenodo.org/records/5151941/files/MARIDA.zip?download=1"
data_zip = "marida.zip"
marida_scenes_path = "patches"
gdown.download(data_url, data_zip)
!unzip -q marida.zip

Downloading...
From: https://zenodo.org/records/5151941/files/MARIDA.zip?download=1
To: /kaggle/working/marida.zip
100%|██████████| 1.16G/1.16G [18:49<00:00, 1.03MB/s]


In [3]:
class MARIDA(Dataset):
    def __init__(self, mode = 'train', transform=None, standardization=None, path ="", agg_to_water= True):
        if mode in ["train", "test", "val"]:
            self.patches = np.genfromtxt(os.path.join(path, "splits", f"{mode}_X.txt"), dtype="str")
        else:
            raise Exception(f"invali split mode: {mode}")

        self._images = []
        self._masks = []

        for patch in tqdm(self.patches, desc = f"Loading {mode} samples"):
            
            patch_folder = '_'.join(['S2'] + patch.split('_')[:-1])
            patch_name = '_'.join(['S2'] + patch.split('_'))
            image_file = os.path.join(path, 'patches', patch_folder, f"{patch_name}.tif")
            mask_file = os.path.join(path, 'patches', patch_folder,f"{patch_name}_cl.tif")
            
            ds = gdal.Open(mask_file)
            temp = np.copy(ds.ReadAsArray().astype(np.int64))

            
            if agg_to_water:
                temp[temp==15]=7          # Mixed Water to Marine Water Class
                temp[temp==14]=7          # Wakes to Marine Water Class
                temp[temp==13]=7          # Cloud Shadows to Marine Water Class
                temp[temp==12]=7          # Waves to Marine Water Class

            # setting non-labeled pixels to -1 in the mask
            temp = np.copy(temp - 1)
            ds=None
            
            self._masks.append(temp)
            
            ds = gdal.Open(image_file)
            temp = np.copy(ds.ReadAsArray())
            ds=None
            self._images.append(temp)          

        self.impute_nan = np.tile(bands_mean, (temp.shape[1],temp.shape[2],1))
        self.mode = mode
        self.transform = transform
        self.standardization = standardization
        self.path = path
        self.agg_to_water = agg_to_water
        
    def __len__(self):
        return len(self._images)
    
    def getnames(self):
        return self.patches
    
    def __getitem__(self, idx):       
        image = self._images[idx]
        mask = self._masks[idx]

        image = np.moveaxis(image, [0, 1, 2], [2, 0, 1]).astype(np.float32)
        
        nan_mask = np.isnan(image)
        image[nan_mask] = self.impute_nan[nan_mask]
        
        if self.transform is not None:
            mask = mask[:,:,np.newaxis]
            # applying the same rotation on the image-mask pair
            stack = np.concatenate([image, mask], axis=-1).astype(np.float32) 

            stack = self.transform(stack) # h, w, c => c, h, w

            image = stack[:-1,:,:]
            mask = stack[-1,:,:].long()


        # print(image)
        
        # if not isinstance(image, torch.Tensor):
        #     image = torch.from_numpy(image)

        
        if self.standardization is not None:
            image = self.standardization(image)

        return image, mask
    
class RandomRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return vF.rotate(x, angle)
    
def gen_weights(class_distribution, c = 1.02):
    return 1/torch.log(c + class_distribution)

In [4]:
class_distr = torch.tensor([0.00452, 0.00203, 0.00254, 0.00168, 0.00766, 0.15206, 0.20232,
 0.35941, 0.00109, 0.20218, 0.03226, 0.00693, 0.01322, 0.01158, 0.00052], device=device)

bands_mean = np.array([0.05197577, 0.04783991, 0.04056812, 0.03163572, 0.02972606, 0.03457443,
 0.03875053, 0.03436435, 0.0392113,  0.02358126, 0.01588816]).astype(np.float32)

bands_std = np.array([0.04725893, 0.04743808, 0.04699043, 0.04967381, 0.04946782, 0.06458357,
 0.07594915, 0.07120246, 0.08251058, 0.05111466, 0.03524419]).astype(np.float32)

transformTrain = transforms.Compose([transforms.ToTensor(),
                                    RandomRotationTransform([-90, 0, 90, 180]),
                                    transforms.RandomHorizontalFlip()])
    
transformTest = transforms.Compose([transforms.ToTensor()])
    
standardization = transforms.Normalize(bands_mean, bands_std)

datasetTrain = MARIDA('train', transform=transformTrain, standardization = standardization)
datasetTest = MARIDA('val', transform=transformTest, standardization = standardization)

trainLoader = DataLoader(datasetTrain,
                        batch_size=batch_size, 
                        shuffle=True
                        )


testLoader = DataLoader(datasetTest, 
                        batch_size=batch_size, 
                        shuffle=False,
                        )


agg_distr = sum(class_distr[-4:])
class_distr[6] += agg_distr
class_distr = class_distr[:-4]

weight = gen_weights(class_distr, c = 1.03).to(device)

Loading val samples: 100%|██████████| 328/328 [00:02<00:00, 140.03it/s]


In [5]:
weight.size(), class_distr.size()

(torch.Size([11]), torch.Size([11]))

In [6]:
def conv3x3(in_channels, out_channels, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, channels, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.mlp = nn.Sequential(nn.Conv2d(channels, channels // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(channels // 16, channels, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class ResidualBlock(nn.Module):
    def __init__(self, inputChannel, outputChannel, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(inputChannel, outputChannel, stride)
        self.bn1 = nn.BatchNorm2d(outputChannel)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(outputChannel, outputChannel)
        self.bn2 = nn.BatchNorm2d(outputChannel)
        self.downsample = downsample
        self.ca = ChannelAttention(outputChannel)
        self.sa = SpatialAttention()
        
    # def forward(self, x):
    #     residual = x
    #     out = self.conv1(x)
    #     out = self.bn1(out)
    #     out = self.relu(out)
    #     out = self.conv2(out)
    #     out = self.bn2(out)
    #     if self.downsample:
    #         residual = self.downsample(x)
    #     out += residual
    #     out = self.relu(out)
    #     caOutput = self.ca(out)
    #     out = caOutput * out
    #     saOutput = self.sa(out)
    #     out = saOutput * out
    #     return out, saOutput

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.bn2(out)
        out = self.relu(out)
        caOutput = self.ca(out)
        out = caOutput * out
        saOutput = self.sa(out)
        out = saOutput * out
        return out, saOutput


class DownSampleWithAttention(nn.Module):
    def __init__(self, inputChannel, outputChannel):
        super().__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(inputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2),
            nn.Conv2d(outputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2),
            nn.AvgPool2d(2)
        )
        self.ca = ChannelAttention(outputChannel)
        self.sa = SpatialAttention()
    
    def forward(self,x):
        x = self.convolution(x)
        caOutput = self.ca(x)
        x = caOutput * x
        saOutput = self.sa(x)
        x = saOutput * x
        return x, saOutput

    
class UpSampleWithAttention(nn.Module):
    def __init__(self, inputChannel, outputChannel):
        super().__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(inputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2),
            nn.Conv2d(outputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2)
        )
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.ca = ChannelAttention(outputChannel)
        self.sa = SpatialAttention()
    
    def forward(self, x):
        x = self.upsample(x)
        x = self.convolution(x)
        caOutput = self.ca(x)
        x = caOutput * x
        saOutput = self.sa(x)
        x = saOutput * x
        return x, saOutput

class ResidualAttentionUNet(nn.Module):
  def __init__(self, inputChannel, outputChannel):
    super().__init__()
    self.downsample1 = DownSampleWithAttention(inputChannel, 32)
    self.downsample2 = DownSampleWithAttention(32, 64)
    self.downsample3 = DownSampleWithAttention(64, 128)
    self.downsample4 = DownSampleWithAttention(128, 256)
    self.downsample5 = DownSampleWithAttention(256, 512)

    self.residualBlock1 = ResidualBlock(512, 512)
    self.residualBlock2 = ResidualBlock(512, 512)
    self.residualBlock3 = ResidualBlock(512, 512)

    self.upsample1 = UpSampleWithAttention(512, 256)
    self.upsample2 = UpSampleWithAttention(512, 128)
    self.upsample3 = UpSampleWithAttention(256, 64)
    self.upsample4 = UpSampleWithAttention(128, 32)
    self.upsample5 = UpSampleWithAttention(64, 32)
    self.classification = nn.Sequential(
            nn.Conv2d(32, outputChannel, kernel_size=1),
        )

  def forward(self, x):
    scale128, sa128down = self.downsample1(x)
    scale64, sa64down = self.downsample2(scale128)
    scale32, sa32down = self.downsample3(scale64)
    scale16, sa64down = self.downsample4(scale32)
    scale8, sa8down = self.downsample5(scale16)
    scale8, sa8down = self.residualBlock1(scale8)
    scale8, sa8down = self.residualBlock2(scale8)
    scale8, sa8down = self.residualBlock3(scale8)
    upscale16, sa16up = self.upsample1(scale8)
    upscale16 = torch.cat([upscale16, scale16], dim=1)
    upscale32, sa32up = self.upsample2(upscale16)
    upscale32 = torch.cat([upscale32, scale32], dim=1)
    upscale64, sa64up = self.upsample3(upscale32)
    upscale64 = torch.cat([upscale64, scale64], dim=1)
    upscale128, sa128up = self.upsample4(upscale64)
    upscale128 = torch.cat([upscale128, scale128], dim=1)
    upscale256, sa256up = self.upsample5(upscale128)
    finaloutput = self.classification(upscale256)
    return finaloutput

In [7]:
model = ResidualAttentionUNet(11, 11).to(device)

In [8]:
# class_distr = torch.tensor([0.0001, *class_distr]).to(device)
model.classification[0].weight.size()

torch.Size([11, 32, 1, 1])

In [9]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='mean', weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=8e-4, weight_decay=1e-2)

# assuming about 40 reductions => .9 ** 40 = 1e-2, starting from 8e-4 ending with 8e-6
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=5)

In [10]:
def Evaluation(y_predicted, y_true):

    micro_prec = precision_score(y_true, y_predicted, average='micro')
    macro_prec = precision_score(y_true, y_predicted, average='macro')
    weight_prec = precision_score(y_true, y_predicted, average='weighted')
    
    micro_rec = recall_score(y_true, y_predicted, average='micro')
    macro_rec = recall_score(y_true, y_predicted, average='macro')
    weight_rec = recall_score(y_true, y_predicted, average='weighted')
        
    macro_f1 = f1_score(y_true, y_predicted, average="macro")
    micro_f1 = f1_score(y_true, y_predicted, average="micro")
    weight_f1 = f1_score(y_true, y_predicted, average="weighted")
        
    subset_acc = accuracy_score(y_true, y_predicted)
    
    iou_acc = jaccard_score(y_true, y_predicted, average='macro')

    info = {
            "macroPrec" : macro_prec,
            "microPrec" : micro_prec,
            "weightPrec" : weight_prec,
            "macroRec" : macro_rec,
            "microRec" : micro_rec,
            "weightRec" : weight_rec,
            "macroF1" : macro_f1,
            "microF1" : micro_f1,
            "weightF1" : weight_f1,
            "subsetAcc" : subset_acc,
            "IoU": iou_acc
            }
    
    return info

In [11]:
# logits.size(), target.size(), class_distr.size(), weight.size()

In [13]:
metrics_history = []
epochs = 160
for epoch in range(1, epochs+1):
    model.train()
    pb = tqdm(trainLoader, desc=f"epoch {epoch}/{epochs}: ")
    yTrue = []
    yPredicted = []

    bg_yTrue = []
    bg_yPredicted = []
    for image, target in pb:
        image, target = image.to(device), target.to(device)
        optimizer.zero_grad()

        logits = model(image)
        
        loss = criterion(logits, target)

        loss.backward()
        optimizer.step()
        pb.set_postfix(loss=loss.item())

        if epoch % 10 == 0:
            with torch.no_grad():
                logits = logits.detach()
                logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
                logits = logits.reshape((-1,11))
                target = target.reshape(-1)
                ###################################################################################
                mask = target != -1
                ###################################################################################
                
                # bg_logits = logits[~mask]
                # bg_target = target[~mask]
    
                # only considering annotated pixels
                logits = logits[mask]
                target = target[mask]
    
                probs = F.softmax(logits, dim=1).cpu().numpy()
                target = target.cpu().numpy()
                yPredicted += probs.argmax(1).tolist()
                yTrue += target.tolist()
        
                
                # bg_probs = torch.nn.functional.softmax(bg_logits, dim=1).cpu().numpy()
                # bg_target = bg_target.cpu().numpy()
                
                # bg_yPredicted += bg_probs.argmax(1).tolist()
                # bg_yTrue += bg_target.tolist()


    if epoch % 10 == 0:
        yPredicted = np.asarray(yPredicted)
        yTrue = np.asarray(yTrue)
        acc = Evaluation(yPredicted, yTrue)
        print(acc)
    
        # bg_yPredicted = np.asarray(bg_yPredicted)
        # bg_yTrue = np.asarray(bg_yTrue)
        # bg_acc = Evaluation(bg_yPredicted, bg_yTrue)
        # print("background:", bg_acc)


    model.eval()
    yTrue = []
    yPredicted = []
    testLossF = []
    
    # bg_yTrue = []
    # bg_yPredicted = []
    with torch.no_grad():
        for image, target in testLoader:
            image, target = image.to(device), target.to(device)
            logits = model(image)
            loss = criterion(logits, target)

            logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
            logits = logits.reshape((-1,11))
            target = target.reshape(-1)
            ###################################################################################
            mask = target != -1
            ###################################################################################
            
            # bg_logits = logits[~mask]
            # bg_target = target[~mask]
            
            logits = logits[mask]
            target = target[mask]
            

            probs = F.softmax(logits, dim=1).cpu().numpy()
            target = target.cpu().numpy()
            # testBatches += target.shape[0]
            testLossF.append((loss.data*target.shape[0]).tolist())
            yPredicted += probs.argmax(1).tolist()
            yTrue += target.tolist()


            # bg_probs = torch.nn.functional.softmax(bg_logits, dim=1).cpu().numpy()
            # bg_target = bg_target.cpu().numpy()

            # bg_yPredicted += bg_probs.argmax(1).tolist()
            # bg_yTrue += bg_target.tolist()
        
        yPredicted = np.asarray(yPredicted)
        yTrue = np.asarray(yTrue)
        acc = Evaluation(yPredicted, yTrue)
        metrics_history.append(acc)

        # bg_yPredicted = np.asarray(bg_yPredicted)
        # bg_yTrue = np.asarray(bg_yTrue)
        # bg_acc = Evaluation(bg_yPredicted, bg_yTrue)
        print(acc)
        # print("background:", bg_acc)
    scheduler.step(sum(testLossF) / len(testLoader.dataset))

epoch 1/160: 100%|██████████| 22/22 [00:20<00:00,  1.05it/s, loss=2.14]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'macroPrec': 0.4267384511353999, 'microPrec': 0.9347120158421789, 'weightPrec': 0.9477184183697129, 'macroRec': 0.44987994177704427, 'microRec': 0.9347120158421789, 'weightRec': 0.9347120158421789, 'macroF1': 0.40161881912989283, 'microF1': 0.9347120158421789, 'weightF1': 0.9355559099533824, 'subsetAcc': 0.9347120158421789, 'IoU': 0.34647244520972104}


epoch 2/160: 100%|██████████| 22/22 [00:21<00:00,  1.05it/s, loss=2.02]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'macroPrec': 0.45352425264758867, 'microPrec': 0.9476870231156911, 'weightPrec': 0.9454809140069043, 'macroRec': 0.4809397433030689, 'microRec': 0.9476870231156911, 'weightRec': 0.9476870231156911, 'macroF1': 0.4510131572804308, 'microF1': 0.947687023115691, 'weightF1': 0.9447634607297276, 'subsetAcc': 0.9476870231156911, 'IoU': 0.4042439744911324}


epoch 3/160: 100%|██████████| 22/22 [00:21<00:00,  1.04it/s, loss=1.58] 
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

{'macroPrec': 0.7787390807401017, 'microPrec': 0.9634822760931385, 'weightPrec': 0.9749645206597927, 'macroRec': 0.8393817246707941, 'microRec': 0.9634822760931385, 'weightRec': 0.9634822760931385, 'macroF1': 0.7903571401728761, 'microF1': 0.9634822760931385, 'weightF1': 0.9680094789319618, 'subsetAcc': 0.9634822760931385, 'IoU': 0.7059558676757532}


epoch 159/160: 100%|██████████| 22/22 [00:21<00:00,  1.00it/s, loss=0.19]  


{'macroPrec': 0.7741951351428042, 'microPrec': 0.9590571651134199, 'weightPrec': 0.9732205666080511, 'macroRec': 0.8331800622106097, 'microRec': 0.9590571651134199, 'weightRec': 0.9590571651134199, 'macroF1': 0.7825546217571837, 'microF1': 0.9590571651134199, 'weightF1': 0.9644150381686963, 'subsetAcc': 0.9590571651134199, 'IoU': 0.6932280669228477}


epoch 160/160: 100%|██████████| 22/22 [00:22<00:00,  1.00s/it, loss=0.0858]


{'macroPrec': 0.8645190486917859, 'microPrec': 0.9540092032826284, 'weightPrec': 0.9544440325954648, 'macroRec': 0.8963173580246551, 'microRec': 0.9540092032826284, 'weightRec': 0.9540092032826284, 'macroF1': 0.8775310175201606, 'microF1': 0.9540092032826284, 'weightF1': 0.9541154965557018, 'subsetAcc': 0.9540092032826284, 'IoU': 0.7926797300827197}
{'macroPrec': 0.7982900714638343, 'microPrec': 0.9731771639871987, 'weightPrec': 0.974949127343406, 'macroRec': 0.8394420265747445, 'microRec': 0.9731771639871987, 'weightRec': 0.9731771639871987, 'macroF1': 0.8138103610502234, 'microF1': 0.9731771639871987, 'weightF1': 0.9737532443861577, 'subsetAcc': 0.9731771639871987, 'IoU': 0.721373018646708}


In [14]:
test_dataset = MARIDA('test', transform=transformTest, standardization = standardization)

test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=False
                        )

Loading test samples: 100%|██████████| 359/359 [00:02<00:00, 140.26it/s]


In [16]:
image.size()

torch.Size([11, 256, 256])

In [17]:
model.eval()
yTrue = []
yPredicted = []
testLossF = []

with torch.no_grad():
    for image, target in test_loader:
        image, target = image.to(device), target.to(device)
        logits = model(image)
        loss = criterion(logits, target)

        logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
        logits = logits.reshape((-1,11))
        target = target.reshape(-1)
        ###################################################################################
        mask = target != -1
        ###################################################################################
        
        logits = logits[mask]
        target = target[mask]
        

        probs = F.softmax(logits, dim=1).cpu().numpy()
        target = target.cpu().numpy()
        testLossF.append((loss.data*target.shape[0]).tolist())
        yPredicted += probs.argmax(1).tolist()
        yTrue += target.tolist()
    
    yPredicted = np.asarray(yPredicted)
    yTrue = np.asarray(yTrue)
    acc = Evaluation(yPredicted, yTrue)
    print(acc)

{'macroPrec': 0.8308193508991121, 'microPrec': 0.9449767272391373, 'weightPrec': 0.9499374229685424, 'macroRec': 0.8138374287642378, 'microRec': 0.9449767272391373, 'weightRec': 0.9449767272391373, 'macroF1': 0.8133310501555545, 'microF1': 0.9449767272391373, 'weightF1': 0.9456891538974435, 'subsetAcc': 0.9449767272391373, 'IoU': 0.7062209203051165}


In [19]:
pd.DataFrame(acc.items(), columns=['Metric', 'Value'])

,Metric,Value
0,macroPrec,0.830819
1,microPrec,0.944977
2,weightPrec,0.949937
3,macroRec,0.813837
4,microRec,0.944977
5,weightRec,0.944977
6,macroF1,0.813331
7,microF1,0.944977
8,weightF1,0.945689
9,subsetAcc,0.944977
